In [1]:
import torch
import torch.nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import InMemoryDataset, Data
import torch_geometric.transforms as T
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

import networkx as nx
from sklearn.preprocessing import StandardScaler


In [22]:
G = nx.karate_club_graph()
labels = np.asarray([G.nodes[i]['club'] != 'Mr.Hi' for i in G.nodes]).astype(np.int64)
print(len(labels)) #34 nodes
print(len(np.asarray(G.nodes)))

adj = nx.to_scipy_sparse_matrix(G).tocoo() #adjacency matrix
row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long) #first entry
col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long) #second entry
edge_index = torch.stack([row, col], dim=0) #both into one

# print(row)
# print(col)
# print(adj)

print(edge_index) #combineboth row and col matrices

empty = np.zeros((len(labels), len(labels)))
# print(empty)

embeddings = np.array(list(dict(G.degree()).values())) #degree is total node neighbours/connections (ie. node 0 has 16 connections)
print(embeddings)

scale = StandardScaler() #normalize and scale degree values
embeddings = scale.fit_transform(embeddings.reshape(-1, 1))
print(embeddings)

34
34
tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,
          3,  3,  3,  3,  3,  4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,
          7,  7,  8,  8,  8,  8,  8,  9,  9, 10, 10, 10, 11, 12, 12, 13, 13, 13,
         13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 19, 20, 20, 21,
         21, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 27, 27,
         27, 27, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 31, 31,
         31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33,
         33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33],
        [ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 17, 19, 21, 31,  0,  2,
          3,  7, 13, 17, 19, 21, 30,  0,  1,  3,  7,  8,  9, 13, 27, 28, 32,  0,
          1,  2,  7, 12, 13,  0,  6, 10,  0,  6, 10, 16,  0,  4,  5, 16,  0,  1,
          2,  3,  0,  2, 30, 32, 33,  2, 33, 

In [ ]:
class KarateDataset(InMemoryDataset):
    def __init__ (self, root, transform=None):
        super(KarateDataset, self).__init__('.', transform, None, None)
        data = Data(edge_index=edge_index)

        data.num_nodes = G.number_of_nodes()
        data.x = torch.from_numpy(embeddings).type(torch.float32)
        y = torch.from_numpy(labels).type(torch.long)     